In [ ]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

In [ ]:
geolocator = Nominatim(user_agent="UrbanTopologyAnalysisService")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [ ]:
addresses = pd.read_csv("../../data/raw data/Нежилые_объекты.csv", index_col=1)

In [ ]:
addresses

### Data Refactor

In [ ]:
def refactor_street_apply(street):
    res = ""
    list_addresses = street.split(",")
    if (len(list_addresses) > 1):
        list_addresses.reverse()
        list_addresses = [string.strip() for string in list_addresses]
        res = ", ".join(list_addresses)
    else:
        res = list_addresses[0]
    return res

In [ ]:
addresses["Улица"] = addresses["Улица"].progress_apply(refactor_street_apply)

In [ ]:
def refactor_house_apply(house):
    if type(house) != str:
        return " "
    else:
        return house

In [ ]:
addresses["Дом"] = addresses["Дом"].progress_apply(refactor_house_apply)

In [ ]:
data_with_geo = pd.read_csv("../../data/result/commercial.csv", index_col=0)
data_with_geo

In [ ]:
new_data = pd.DataFrame({"Адрес": [], "Геопозиция": [], "Площадь": []})

for i in range(addresses.shape[0]-1, -1, -1):
  street = str(addresses.iloc[i]["Дом"]) + ", " + addresses.iloc[i]["Улица"] + ", Санкт-Петербург"
  exist_in_geo_data = data_with_geo.loc[data_with_geo["Адрес"] == street, "Геопозиция"].array
  row = [
          street,
          exist_in_geo_data[0] if len(exist_in_geo_data) != 0 else None,
          addresses.iloc[i]["Площадь объекта"]
        ]
  new_data = pd.concat([pd.DataFrame([row], columns=new_data.columns), new_data], ignore_index=True)
    
# new_data = new_data.drop_duplicates()
new_data

In [ ]:
new_data.loc[new_data["Геопозиция"].isnull()]

In [ ]:
def apply_geocode(street):
    location = geocode(street)
    return (location.latitude, location.longitude) if location else None
new_data["Геопозиция"] = new_data["Адрес"].progress_apply(apply_geocode)

In [ ]:
def apply_add_lat(geop):
    return geop[1:-1].split(",")[0]

def apply_add_lon(geop):
    return geop[1:-1].split(",")[1]

new_data["longitude"] = new_data["Геопозиция"].apply(apply_add_lon)
new_data

In [ ]:
new_data.to_csv("../../data/result/commercial_with_area.csv")

In [ ]:
null_new_data = new_data.loc[new_data["Геопозиция"].isnull()]
null_new_data

In [ ]:
null_new_data.to_csv("./null.csv")

In [ ]:
def apply_geocode(street):
    location = geocode(street)
    return (location.latitude, location.longitude) if location else None
new_data.loc[new_data["Геопозиция"].isnull(), "Геопозиция"] = new_data.loc[new_data["Геопозиция"].isnull(), "Адрес"].progress_apply(apply_geocode)

### Data analysis

In [ ]:
streets = addresses["Улица"].to_list()

In [ ]:
streets

In [ ]:
big_streets = []
for street in streets:
    if len(street.split(",")) > 1:
        big_streets.append(street)

print(len(big_streets))

In [ ]:
first_part_big_streets = []
for street in big_streets:
    first_part_big_streets.append(street.split(",")[0])

first_part_big_streets = list(set(first_part_big_streets))
print(first_part_big_streets)

In [ ]:
standart_streets = []
for street in streets:
    if len(street.split(",")) == 1:
        standart_streets.append(street)

print(len(standart_streets))

In [ ]:
null_houses = addresses.loc[addresses["Дом"].isnull()]

In [ ]:
print(null_houses.shape[0])

In [ ]:
null_houses

### Take data for one area

In [ ]:
data_for_viborg = pd.DataFrame({"Адрес": [], "Геопозиция": []})
viborg_residents = addresses.loc[addresses["Район"] == "Выборгский"]

for i in range(viborg_residents.shape[0]):
    row = [
            str(viborg_residents.iloc[i]["Дом"]) + ", " + viborg_residents.iloc[i]["Улица"] + ", Санкт-Петербург",
            None
          ]
    data_for_viborg = pd.concat([pd.DataFrame([row], columns=data_for_viborg.columns), data_for_viborg], ignore_index=True)

In [ ]:
data_for_viborg

In [ ]:
data_for_viborg = data_for_viborg.iloc[:60]

In [ ]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
data_for_viborg["Геопозиция"] = data_for_viborg["Адрес"].progress_apply(geocode)
data_for_viborg["Геопозиция"] = data_for_viborg["Геопозиция"].apply(lambda loc: (loc.latitude, loc.longitude) if loc else None)

In [ ]:
data_for_viborg.to_csv("Viborg.csv")

In [ ]:
null_in_viborg = data_for_viborg.loc[data_for_viborg["Геопозиция"].isnull()]

In [ ]:
null_in_viborg

In [ ]:
def apply_geocode(street):
    location = geocode(street)
    return (location.latitude, location.longitude) if location else None
data_for_viborg.loc[data_for_viborg["Геопозиция"].isnull(), "Геопозиция"] = data_for_viborg.loc[data_for_viborg["Геопозиция"].isnull(), "Адрес"].progress_apply(apply_geocode)

#### Krasnoselsky

In [ ]:
data_for_krasnoselsky = pd.DataFrame({"Адрес": [], "Геопозиция": []})
krasnoselsky_residents = addresses.loc[addresses["Район"] == "Красносельский"]

for i in range(krasnoselsky_residents.shape[0]):
    row = [
            str(krasnoselsky_residents.iloc[i]["Дом"]) + ", " + krasnoselsky_residents.iloc[i]["Улица"] + ", Санкт-Петербург",
            None
          ]
    data_for_krasnoselsky = pd.concat([pd.DataFrame([row], columns=data_for_krasnoselsky.columns), data_for_krasnoselsky], ignore_index=True)

data_for_krasnoselsky

In [ ]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
data_for_krasnoselsky["Геопозиция"] = data_for_krasnoselsky["Адрес"].progress_apply(geocode)
data_for_krasnoselsky["Геопозиция"] = data_for_krasnoselsky["Геопозиция"].apply(lambda loc: (loc.latitude, loc.longitude) if loc else None)
data_for_krasnoselsky.to_csv("Krasnoselsky.csv")

In [ ]:
null_in_krasnoselsky = data_for_krasnoselsky.loc[data_for_krasnoselsky["Геопозиция"].isnull()]
null_in_krasnoselsky